# Sewershed delineation

In this Jupyter notebook for Python, we delineate a sewershed based on a sewer network and census blocks.

## Data requirements

- Sewer mains
  * Sewer gravity mains and possibly force mains.
  * All flowing into one point (head of the sewershed).
  * The lines don't need to be spatially connected in the data, but the processing will assume that all lines flow into a single point.
- US Census blocks as polygons (areas) with demographics as attributes


<div class="alert alert-block alert-warning">
<b>Note:</b>
GRASS GIS 8.4 (currently in preview) is need for new version of <em>v.dissolve</em> tool and new <em>v.fill.holes</em> tool. Available at <a href="https://grass.osgeo.org/download/">grass.osgeo.org, under download</a>.
</div>

## Software setup 

We will use a couple of standard Python packages and GRASS GIS.

For now, the setup here assumes Linux. Instructions for Windows are available at [GRASS GIS Jupyter notebooks wiki page](https://grasswiki.osgeo.org/wiki/GRASS_GIS_Jupyter_notebooks#Running_a_Jupyter_notebook_locally).

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
from pathlib import Path

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

 extension appropriate for the format, e.g. mains.shp. Census blocks should be in data/census directory and named blocks plus a file extension. Either rename the files or modify the code below if ne extension appropriate for the format, e.g. mains.shp. Census blocks should be in data/census directory and named blocks plus a file extension. Either rename the files or modify the code below if ne## Get the data ready

This notebooks needs gravity mains (or all mains) as vector lines and census blocks as vector polygons with attributes. The gravity mains file should be in directory `data/sewers` and should be named `mains` with file extension appropriate for the format, e.g. `mains.shp`. Census blocks should be in `data/census` directory and named `blocks` plus a file extension. Either rename the files or modify the code below if needed.

The paths can be not only directories but also ZIP files. Similarly, the files can also be layers. The names will be passed to GDAL.

In [ ]:
data_directory = "data/sewers_state"
point_file = "NCWMN_shed_pts_042523"  # Filename without file extension

In [ ]:
grass_project = "data/state_overview"

To compute the data, we will use a GRASS project (aka location).

In [ ]:
!grass -e -c $data_directory $grass_project

In [ ]:
session = gj.init(grass_project)

We will use vector lines of gravity mains as our sewershed network. Here, we are using Raleigh as an example. We will also use the US 2020 census blocks for North Carolina. The census blocks are polygons (i.e., areas).

We store the names of GRASS vector maps in Python variables and will use the variables from now on.

In [ ]:
point_vector = "points"

In [ ]:
gs.run_command("v.import", input=data_directory, layer=point_file, output=point_vector)

To limit the number of census blocks we need to import, we set the computational region to the extent of sewer mains (extended in all directions by 100 map units - feet or meter).

In [ ]:
gs.run_command("g.region", vector=point_vector, res=1, grow=100)

## Select census blocks

To get only the relevant census blocks, we select only census blocks which overlap with the sewer network.

In [ ]:
large_sewersheds = "large_sewersheds"

gs.run_command(
    "v.extract",
    input=point_vector,
    output=large_sewersheds,
    where="pop_2020 > 100000",
)

In [ ]:
gs.run_command("g.region", vector="nc_state", res=1000, grow=50)
state_map = gj.Map(width=1024, use_region=True)
state_map.d_background(color="white")
state_map.d_vect(flags="s", map="nc_state", fill_color="none")
state_map.d_vect_thematic(
    map=point_vector,
    column="pop_2020",
    legend_title="Population per sewershed",
    colors="#ADA8A7,#6A00A8,#B12A90,#E16462,#FCA636,#F0F921",
    breaks=[100000, 200000, 300000, 400000],
    boundary_color="none",
    icon="basic/circle",
    size=10,
)
# state_map.d_vect(flags="s", map=point_vector, where="pop_2020 > 100000", color="none", fill_color="none", attribute_column="site_name", label_color="#777777", label_size=10)
gs.run_command(
    "v.label.sa",
    map=large_sewersheds,
    column="site_name",
    labels="points_site_name",
    font="DejaVuSans",
    size=30000,
    color="black",
    isize=25000,
)
state_map.d_labels(labels="points_site_name")
state_map.d_legend_vect(flags="b", at=(3, 34), columns=2)
state_map.d_barscale(flags="n", at=(2, 7), length=200, units="kilometers")
state_map.save(Path(sewers_directory) / "nc_sewersheds.png")
state_map.show()